## 多输入通道和多输出通道
---

### 多输入通道

In [1]:
import torch
from torch import nn
import sys
sys.path.append('..')
import d2lzh_pytorch as d2l

In [2]:
# d2l.corr2d 二维卷积运算
def corr2d_multi_in(X, K):
    res = d2l.corr2d(X[0, :, :], K[0, :, :])
    for i in range(1, X.shape[0]):
        res += d2l.corr2d(X[i, :, :], K[i, :, :])
    return res

In [15]:
X = torch.tensor([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = torch.tensor([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])
print(X)
print(X[0, :, :])
print(X.shape) # 解释：[矩阵个数，行，列]

tensor([[[0, 1, 2],
         [3, 4, 5],
         [6, 7, 8]],

        [[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]])
tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
torch.Size([2, 3, 3])


In [5]:
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [17]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K])

In [21]:
K = torch.stack([K, K + 1, K + 2]) # 设置三个准备输出的通道
# print(K)
print(K.shape) # 划分为 [3..., 3, 2] 行列[2, 2]

torch.Size([3, 3, 3, 2, 2, 2])


In [19]:
corr2d_multi_in_out(X, K)  # 分别是 K , K+1 , K+2结果

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

### 1 × 1 卷积层
![1mul1conv2d](./img/Snipaste_2021-09-27_10-39-46.png)
输入通道数为3、输出通道数为2的1×11×1卷积核的互相关计算

1×1卷积层通常用来调整网络层之间的通道数，并控制模型复杂度

In [29]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.view(c_i, h * w)
    K = K.view(c_o, c_i)
    Y = torch.mm(K, X)
    return Y.view(c_o, h, w)

In [32]:
X = torch.rand(3, 3, 3) # c_i = 3  ==> [3, 9]
K = torch.rand(2, 3, 1, 1) # c_o = 2  ==> [2, 3]
print('X: ', X.view(3,9))
print('K: ', K.view(2,3))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
print(Y1)
print(Y2)

(Y1 - Y2).norm().item() < 1e-6

X:  tensor([[0.6542, 0.5544, 0.1346, 0.2838, 0.1291, 0.8641, 0.4491, 0.5873, 0.0208],
        [0.6586, 0.1975, 0.6296, 0.0510, 0.4998, 0.3774, 0.8282, 0.5707, 0.4979],
        [0.3851, 0.1948, 0.3365, 0.2283, 0.0590, 0.4665, 0.5523, 0.4093, 0.3155]])
K:  tensor([[0.1657, 0.6026, 0.5720],
        [0.4236, 0.6342, 0.9604]])
123： 3
tensor([[[0.7256, 0.3223, 0.5942],
         [0.2083, 0.3563, 0.6374],
         [0.8894, 0.6754, 0.4840]],

        [[1.0647, 0.5472, 0.7795],
         [0.3718, 0.4283, 1.0533],
         [1.2459, 1.0038, 0.6276]]])
tensor([[[0.7256, 0.3223, 0.5942],
         [0.2083, 0.3563, 0.6374],
         [0.8894, 0.6754, 0.4840]],

        [[1.0647, 0.5472, 0.7795],
         [0.3718, 0.4283, 1.0533],
         [1.2459, 1.0038, 0.6276]]])


True